# Landsat Coverage Investigation

https://towardsdatascience.com/downloading-landsat-satellite-images-with-python-a2d2b5183fb7

In [ ]:
import pandas as pd
import geopandas as gpd
from landsatxplore.api import API
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import shapely.wkt
import rasterio
import matplotlib
import matplotlib.pyplot as plt
import math
from shapely.strtree import STRtree
import datetime
import numpy as np
import rasterio
from rasterio.features import rasterize
from tqdm import tqdm

# Settings

In [ ]:
roi_shape = 'shapefiles/50south_excl_argentina_falkand_mid.geojson'

# Functions

## Data Engineering

In [33]:
def preprocess_landsat(df, max_date=''):
    # DE - spatial
    df['spatial_coverage'] = df['spatial_coverage'].apply(lambda x : shapely.wkt.loads(x))
    df['geometry_4326'] = df['spatial_coverage'].copy()
    # DE - Time
    df['start_time'] = df['start_time'].apply(lambda x : str(x).split('.')[0])
    df['start_time'] = pd.to_datetime(df['start_time'], format="%Y-%m-%d %H:%M:%S")
    df['month'] = df['start_time'].dt.month
    df['month_name'] = df['start_time'].dt.month_name()
    df['year'] = df['start_time'].dt.year
    # limit by max date
    if max_date:
        df = df[df['start_time'] < max_date]
    # DE - Filesize by Satellite (https://www.usgs.gov/faqs/what-are-landsat-collection-1-level-1-data-product-file-sizes)
    df['size_compressed_mb'] = df['satellite'].map({8:919,9:919,7:235,5:150,4:150,'LANDSAT_4':150, 'LANDSAT_5':150})
    df['size_uncompressed_mb'] = df['satellite'].map({8:1610,9:1610,7:785,5:500,4:500,'LANDSAT_4':500, 'LANDSAT_5':500})
    # DE - Sun Azumith
    if 'sun_elevation_l0ra' in list(df):
        df['sun_zenith_10ra'] = 90 - df['sun_elevation_l0ra'] 
    elif 'sun_elevation' in list(df):
        df['sun_zenith'] = 90 - df['sun_elevation'] 
    # cloud cover categories - https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data
    df['cloud_cover_category'] = df['cloud_cover'].apply(lambda x : 'Clear (<35%)' if x < 35 else ('Cloudy (>65%)' if x>65 else 'MidClouds (35-65%)'))
    # sat name
    df['sat_id'] = df['satellite'].apply(lambda x : 'Landsat ' + str(x) if 'LAND' not in str(x) else 'Landsat ' + x.split('_')[-1])
    # sat name 4/5 
    # load into geopandas df and filter for AOI
    df = gpd.GeoDataFrame(df, geometry='spatial_coverage', crs="EPSG:4326")
    df = df.to_crs(3031)
    return df

def plot_geometries_on_map(df, extent = (-180, 180, -90, -50)):
    plt.rcParams["figure.figsize"] = [14,6]
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_extent(extent, ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    ax.add_geometries(df.geometry, 
                    crs=ccrs.SouthPolarStereo(), 
                    alpha=0.2)
    ax.gridlines(draw_labels=True)
    plt.show()

# limit by shapefile
def filter_results_with_geojson(df, shape_filename, plot=False, crs=4326):
    gdf_inclusion = gpd.read_file(shape_filename)
    #df = df[df.intersects(gdf_inclusion.geometry.values[0])]
    if crs == 4326:
        df = df[df['geometry_4326'].apply(lambda x : x.intersects(gdf_inclusion.geometry.values[0]))]
    else:
        df = df[df.geometry.apply(lambda x : x.intersects(gdf_inclusion.geometry.values[0]))]
    if plot:
        plt.rcParams["figure.figsize"] = [10,8]
        ax = plt.axes(projection=ccrs.PlateCarree(), title='Product Search Area - 50deg south excl. South America and Falkland Islands')
        ax.add_feature(cartopy.feature.LAND)
        ax.add_feature(cartopy.feature.OCEAN)
        ax.add_geometries(gdf_inclusion.geometry, crs=ccrs.PlateCarree(), alpha=0.7)
        plt.show()
    return df

## Plotting - Timeseries Product Count

In [ ]:

def plot_timeseries_products(df, title='',stack_col='sat_id', date_col='beginposition',count_freq='7D', plot_freq='1M'):
    import seaborn as sns
    sns.set_theme()
    sns.set(rc={'figure.figsize':(10,2)})
    df['round_time'] = df[date_col].dt.round(count_freq)
    c = df[['round_time',stack_col,'entity_id']].groupby(['round_time',stack_col]).count().reset_index()
    c = c.pivot(index='round_time', columns=stack_col, values='entity_id').fillna(0)
    c = c.resample(plot_freq).max()
    ax = c.plot(kind='bar', stacked=True, width=1)
    import matplotlib.ticker as ticker
    ticklabels = ['']
    for i in range(1,len(c.index)):
        ticklabels.append('') if c.index[i].year == c.index[i-1].year else ticklabels.append(c.index[i].year)
    ax.xaxis.set_major_formatter(ticker.FixedFormatter(ticklabels))
    ax.set_xlabel('')
    plt.xticks(rotation = 45, fontsize='10')
    plt.yticks(fontsize='10')
    plt.legend(title='')
    plt.title(title)
    set(ticklabels)
    sns.reset_orig()
    plt.show()

## Plotting - WRS2 Grid Based

In [ ]:
def plot_mgrs_product_count(df, title=''):

    #read in the mgrs tile 
    print('loading tile shapefile...')
    if 'OLI' in df['sensor_id'].iloc[0]:
        # landsat 8 and 9
        if df.iloc[0]['day-night_indicator'] == 'DAY':
            shp = 'shapefiles/WRS2_descending_0_daytime/WRS2_descending.shp'
        elif df.iloc[0]['day-night_indicator'] == 'NIGHT':
            shp = 'shapefiles/WRS2_ascending_0_nighttime/WRS2_ascending.shp'
    if 'ETM' in df['sensor_id'].iloc[0]:
        shp = 'shapefiles/WRS2_descending_0_daytime/WRS2_descending.shp'
    
    df_tiles = gpd.read_file(shp)

    # product count
    print('counting products')
    prod_count = df.groupby(['wrs_path', 'wrs_row'])['landsat_product_id'].count().reset_index()
    tile_count = prod_count.merge(df_tiles, right_on=['PATH','ROW'], left_on=['wrs_path', 'wrs_row'])
    tile_count.set_geometry('geometry')
    tile_count = tile_count.rename(columns={'landsat_product_id':'filecount'})
    tile_count = tile_count.sort_values('filecount')
    crs = ccrs.PlateCarree()
    shape = 1000, 1000
    bounds = crs.boundary.bounds
    east, west, south, north = -180, 180, -90, -50
    transform = rasterio.transform.from_bounds(*bounds, *shape)

    import matplotlib.cm as cm

    minima = tile_count.filecount.min()
    maxima = tile_count.filecount.max()

    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)

    plt.rcParams["figure.figsize"] = [10,8]
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_extent((east, west, south, north+1), ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    count_vals = tile_count.filecount.values
    c = []
    for v in count_vals:
        c.append(mapper.to_rgba(v))
    ax.gridlines(draw_labels=True)
    ax.add_geometries(tile_count.geometry, crs=ccrs.PlateCarree(), alpha=0.5, facecolor=c)
    ax.add_feature(cartopy.feature.COASTLINE)
    plt.colorbar(mapper, label='Count of Products')
    plt.title(title)
    plt.show()

def plot_multiple_product_count(df, group, title='', n_cols=2, cbar_label='Count of Products'):

    #read in the mgrs tile 
    print('loading tile shapefile...')
    if df.iloc[0]['day-night_indicator'] == 'DAY':
        df_tiles = gpd.read_file('shapefiles/WRS2_descending_0_daytime/WRS2_descending.shp')
    elif df.iloc[0]['day-night_indicator'] == 'NIGHT':
        df_tiles = gpd.read_file('shapefiles/WRS2_ascending_0_nighttime/WRS2_ascending.shp')
    
    
    # calculate the size of the figure
    n = df[group].nunique()
    n_rows = math.ceil(n/n_cols)

    # using the variable axs for multiple Axes
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_cols,
                        subplot_kw={'projection': ccrs.SouthPolarStereo()},
                        figsize=(n_cols*5,n_rows*4.5))
    
    # product count by group to get max and min for colorbar
    prod_counts = df.groupby([group] + ['wrs_path', 'wrs_row'])['landsat_product_id'].count().reset_index()
    prod_counts = prod_counts.rename(columns={'landsat_product_id':'filecount'})
    minima = prod_counts.filecount.min()
    maxima = prod_counts.filecount.max()
    import matplotlib.cm as cm

    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
    count = 0
    
    plot_grps = df[group].unique()
    try:
        # sort if numeric
        int(plot_grps[0])
        plot_grps = sorted(plot_grps)
    except:
        ...

    for grp in plot_grps:
        grp_df = df[df[group] == grp]
        tile_count = grp_df.groupby([group] + ['wrs_path', 'wrs_row'])['landsat_product_id'].count().reset_index()
        tile_count = tile_count.rename(columns={'landsat_product_id':'filecount'})
        tile_count = tile_count.merge(df_tiles, right_on=['PATH','ROW'], left_on=['wrs_path', 'wrs_row']) 
        tile_count.set_geometry('geometry')
        tile_count = tile_count.sort_values('filecount')
        n_products = tile_count['filecount'].sum()
        east, west, south, north = -180, 180, -90, -50
        r = math.floor((count)/n_cols)
        c = count % n_cols
        ax_i = (r,c) if ((n_cols > 1) and (n_rows>1)) else count # only a single index needed if no cols
        ax[ax_i].set_extent((east, west, south, north), ccrs.PlateCarree())
        ax[ax_i].add_feature(cartopy.feature.LAND)
        ax[ax_i].add_feature(cartopy.feature.OCEAN)
        subtitle = f'{grp} ({n_products:,} products)'
        ax[ax_i].title.set_text(subtitle)
        print(subtitle)
        count_vals = tile_count.filecount.values
        c = []
        for v in count_vals:
            c.append(mapper.to_rgba(v))
        ax[ax_i].add_geometries(tile_count.geometry, crs=ccrs.PlateCarree(), alpha=0.8, facecolor=c)
        ax[ax_i].add_feature(cartopy.feature.COASTLINE)
        gl = ax[ax_i].gridlines(draw_labels=True)
        gl.xlabel_style['rotation']= 0
        gl.xlabel_style['ha']= 'center'
        gl.xlabel_style['va']= 'center'
        count += 1

    # add the colorbar for all plots
    print('Adding colorbar...')
    plt.tight_layout()
    cbar_ax = fig.add_axes([0.05, -0.05, 0.9, 0.04])
    #fig.colorbar(color, cax=cbar_ax, label=cbar_label, orientation="horizontal")
    plt.colorbar(mapper, cax=cbar_ax, label=cbar_label, orientation="horizontal")
    plt.suptitle(title, y=1.02, fontsize='x-large')

    #delete subplot if uneven number
    if count != (n_rows*n_cols):
        if n_cols > 1:
            fig.delaxes(ax[n_rows-1,n_cols-1])
        else:
            fig.delaxes(ax[n])

## Plotting - Point Based Coverage
- This plotting method uses randomly generated points to determine the number of unique passes over a given location. i.e. it accounts for the duplication of data from overlapping products

In [ ]:
def generate_points_in_bounds(filename, number, plot=True): 
    """ function to add a fixed number of points randomly in a polygon
    """  
    from shapely.geometry import Point, Polygon
    import itertools

    #open the spatial AOI
    roi_gdf = gpd.read_file(filename)

    minx, miny, maxx, maxy = roi_gdf.geometry.bounds.values[0]
    xs = np.linspace(minx, maxx, num=number)
    ys = np.linspace(miny, maxy, num=number)
    #scale number for y
    y_number = int(number * (abs(miny-maxy)/abs(minx-maxx)))
    ys = np.linspace(miny, maxy, num=y_number)
    points = list(itertools.product(xs, ys))
    print(len(points))

    # x = np.random.uniform( minx, maxx, number )
    # y = np.random.uniform( miny, maxy, number )
    # points = list(zip(x,y))

    df = pd.DataFrame()
    df['points'] = points
    df['points'] = df['points'].apply(Point)
    points_gdf = gpd.GeoDataFrame(df, geometry='points', crs="EPSG:4326")
    Sjoin = gpd.tools.sjoin(points_gdf, roi_gdf, predicate="within", how='left')
    # # Keep points in "myPoly"
    points_in_poly = Sjoin.intersection(roi_gdf['geometry'].values[0]).to_crs(3031)

    # Plot result
    if plot:
        base = roi_gdf['geometry'].plot(linewidth=1, edgecolor="black")
        points_in_poly.plot(ax=base, linewidth=1, color="black", markersize=8)
        plt.show()
        
    return points_in_poly

def calculate_valid_intersections(df, points_df, min_revisit_time=10, plot=False):
    
    # Build spatial index
    spatial_index = STRtree(df.geometry)
    # Create an empty GeoDataFrame to store the intersections for the current polygon
    intersections = []
    # Iterate over each polygon in the DataFrame
    for p in tqdm(points_df):
        # Find potential intersecting polygons using the spatial index
        potential_intersections = spatial_index.query(p)
        # Check for intersections with potential polygons
        for j in potential_intersections:
             if p.intersects(df.geometry.iloc[j]):
                date = df.start_time.iloc[j]
                sza_col = 'sun_zenith_10ra' if df.satellite.iloc[j] in [8,9] else 'sun_zenith'
                intersections.append({
                    'intersectionpoint':p,
                    'intersectionpointstr':str(p),
                    'intersectionpoint_x':p.x,
                    'intersectionpoint_y':p.y,
                    'start_time': df.start_time.iloc[j],
                    'cloud_cover': df.cloud_cover.iloc[j],
                    'cloud_cover_category': df.cloud_cover_category.iloc[j],
                    'start_time': df.start_time.iloc[j],
                    'land_cloud_cover': df.land_cloud_cover.iloc[j],
                    'scene_cloud_cover': df.scene_cloud_cover.iloc[j],
                    'polygon': df.geometry.iloc[j],
                    'month_name': df.month_name.iloc[j],
                    'year': df.year.iloc[j],
                    'satellite' : df.satellite.iloc[j],
                    'wrs_path': df.wrs_path.iloc[j],
                    'wrs_row': df.wrs_row.iloc[j],
                    'sun_zenith_10ra': df[sza_col].iloc[j],
                })

    #covert to dataframe 
    intersections = pd.DataFrame.from_dict(intersections)

    # sort by point and date aquired
    intersections = intersections.sort_values(by=['intersectionpointstr','start_time'])

    # calculate the days between observations at a given point
    # replace na / first datapoint with placeholder revisit time
    placeholder_dt = datetime.timedelta(days=5)
    intersections['timebetweenobs'] = (intersections['start_time'] - intersections['start_time'].shift(1)).fillna(placeholder_dt)
    
    # The first observation for each point does not have another time to compare
    intersections[intersections['intersectionpointstr']!=intersections['intersectionpointstr'].shift(1)]['timebetweenobs'] = placeholder_dt
    intersections['daysbetweenobs'] = intersections['timebetweenobs'].dt.days.astype(float)
    intersections['minutesbetweenobs'] = intersections['timebetweenobs'].dt.total_seconds().astype(int)/60
    
    # keep only valid revisit times, 
    # i.e. when a point is measure at least 'min_revisit_time' minutes apart
    intersections = intersections[intersections['minutesbetweenobs']>min_revisit_time]
    intersections = gpd.GeoDataFrame(intersections, geometry='intersectionpoint', crs="EPSG:3031")

    # sort by point and date aquired
    intersections = intersections.sort_values(by=['intersectionpointstr','start_time'])

    if plot:
        heatmap, xedges, yedges = np.histogram2d(
        intersections.intersectionpoint_x, 
        intersections.intersectionpoint_y, 
        bins=100)
        extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
        print(extent)

        plt.clf()
        plt.title('Heatmap (not freq count)')
        plt.imshow(heatmap.T, extent=extent, origin='lower')
        plt.colorbar(orientation='horizontal')
        plt.show()

    return intersections

def summarise_intersections(intrsections, group=''):

    if group:
        groupby = ['intersectionpoint',group]
    else:
        groupby = ['intersectionpoint']
    
    intrsection_summary = intrsections.groupby(groupby).agg(
        revisit_count=('intersectionpointstr','count'),
        mean_revisit_mins=('minutesbetweenobs','mean'),
        median_revisit_mins=('minutesbetweenobs','median'),
        mean_cloudcover=('cloud_cover','mean'),
        median_cloudcover=('cloud_cover','median'),
        mean_sza=('sun_zenith_10ra','mean'),
        min_sza=('sun_zenith_10ra','min'),
        max_sza=('sun_zenith_10ra','max'),
    ).reset_index()
    
    return intrsection_summary

def plot_intersection_frequency(intrsections, plot_var, title='', cbar_label='',shape=(1000,1000), cmap='viridis', force_max=''):
    
    # summarise the intersections
    intrsection_summary = summarise_intersections(intrsections)
    
    crs = ccrs.SouthPolarStereo()
    bounds = crs.boundary.bounds
    transform = rasterio.transform.from_bounds(*bounds, *shape)

    freq_raster = np.zeros(shape)
    for i,row in intrsection_summary.iterrows():
        point = row.intersectionpoint
        count = row[plot_var]
        new_rast = rasterize([(point, count)], out_shape=shape, transform=transform)
        # replace the value where higher - i.e. get max value per pixel
        r,c = np.where(new_rast>freq_raster)
        freq_raster[(r,c)] = new_rast[(r,c)]

    # plot the raster
    freq_raster[freq_raster==0] = np.nan
    if force_max:
        freq_raster[freq_raster>force_max] = force_max
    raster = freq_raster #cc_raster if cloud else freq_raster

    crs = ccrs.SouthPolarStereo()
    shape = 1000, 1000
    bounds = crs.boundary.bounds
    east, west, south, north = -180, 180, -90, -50
    transform = rasterio.transform.from_bounds(*bounds, *shape)
    
    plt.rcParams["figure.figsize"] = [10,8]
    ax = plt.axes(projection=ccrs.SouthPolarStereo())
    ax.set_extent((east, west, south, north+1), ccrs.PlateCarree())
    ax.add_feature(cartopy.feature.LAND)
    ax.add_feature(cartopy.feature.OCEAN)
    color = ax.imshow(raster, origin="upper", extent=(bounds[0], bounds[2], bounds[1], bounds[3]), transform=ccrs.SouthPolarStereo(), cmap=cmap)
    ax.add_feature(cartopy.feature.COASTLINE)
    cbar_max = int(raster[raster>-1].max())
    plt.colorbar(color, ticks=np.linspace(0, cbar_max, 10, dtype=int), label=cbar_label)
    ax.gridlines(draw_labels=True)
    plt.title(title)
    plt.show()

def plot_multiple_intersection_frequency(intrsections, plot_var, group, sort_vals='', n_cols=2, title='', cbar_label='',shape=(1000,1000)):
    
    # summarise the intersections
    intrsection_summary = summarise_intersections(intrsections, group=group)
    
    crs = ccrs.SouthPolarStereo()
    bounds = crs.boundary.bounds
    transform = rasterio.transform.from_bounds(*bounds, *shape)

     # calculate the size of the figure
    n = intrsection_summary[group].nunique()
    n_rows = math.ceil(n/n_cols)

    # using the variable axs for multiple Axes
    fig, ax = plt.subplots(nrows=n_rows, ncols=n_cols,
                        subplot_kw={'projection': ccrs.SouthPolarStereo()},
                        figsize=(n_cols*5,n_rows*4.5))
    
    # product count by group to get max and min for colorbar
    minima = intrsection_summary[plot_var].min()
    maxima = intrsection_summary[plot_var].max()

    east, west, south, north = -180, 180, -90, -50
    
    # colorbar
    import matplotlib.cm as cm
    norm = matplotlib.colors.Normalize(vmin=minima, vmax=maxima, clip=True)
    mapper = cm.ScalarMappable(norm=norm, cmap=cm.viridis)
    count = 0

    groups = intrsections[group].unique()
    if 'January' in groups:
        print('Months')
        groups = ['January','February','March','April','May','June','July','August','September','October','November','December']
    else:
        print('Years')
        groups = sorted(groups)
    print(groups)
    for g in tqdm(groups):
        g_intrsection_summary = intrsection_summary[intrsection_summary[group]==g]
        raster = np.zeros(shape)
        for i,row in g_intrsection_summary.iterrows():
            
            # data
            point = row.intersectionpoint
            plot_val = row[plot_var]
            new_rast = rasterize([(point, plot_val)], out_shape=shape, transform=transform)
            
            # replace the value where higher - i.e. get max value per pixel
            r_,c_ = np.where(new_rast>raster)
            raster[(r_,c_)] = new_rast[(r_,c_)]

        raster[raster==0] = np.nan
        r = math.floor((count)/n_cols)
        c = count % n_cols
        ax_i = (r,c) if ((n_cols > 1) and (n_rows>1)) else count # only a single index needed if no cols
        # get product data for each catagory
        #n_products = g_intrsection_summary
        ax[ax_i].set_extent((east, west, south, north), ccrs.PlateCarree())
        ax[ax_i].add_feature(cartopy.feature.LAND)
        ax[ax_i].add_feature(cartopy.feature.OCEAN)
        ax[ax_i].title.set_text(f'{g}')
        color = ax[ax_i].imshow(raster, 
                                origin="upper", 
                                extent=(bounds[0], bounds[2], bounds[1], bounds[3]), 
                                transform=ccrs.SouthPolarStereo(),
                                vmin=minima,
                                vmax=maxima
                                )
        ax[ax_i].add_feature(cartopy.feature.COASTLINE)
        gl = ax[ax_i].gridlines(draw_labels=True)
        gl.xlabel_style['rotation']= 0
        gl.xlabel_style['ha']= 'center'
        gl.xlabel_style['va']= 'center'
        count += 1

    # add the colorbar for all plots
    plt.tight_layout()
    cbar_ax = fig.add_axes([0.05, -0.05, 0.9, 0.04])
    plt.colorbar(mapper, cax=cbar_ax, label=cbar_label, orientation="horizontal")
    plt.suptitle(title, y=1.03, fontsize='x-large')

    #delete subplot if uneven number
    if count != (n_rows*n_cols):
        if n_cols > 1:
            fig.delaxes(ax[n_rows-1,n_cols-1])
        else:
            fig.delaxes(ax[n])


# Generate a set of points for intersection plotting

In [ ]:
N_POINTS = 500
MAX_DATE = datetime.datetime.strptime('16/06/23', '%d/%m/%y')
points_in_roi = generate_points_in_bounds(roi_shape, N_POINTS, plot=True)

# Landsat datasets

| Dataset Name | Dataset ID |
|---|---|
| Landsat 5 TM Collection 2 Level 1 | landsat_tm_c2_l1 |
| Landsat 5 TM Collection 2 Level 2 | landsat_tm_c2_l2 |
| Landsat 7 ETM+ Collection 2 Level 1 | landsat_etm_c2_l1 |
| Landsat 7 ETM+ Collection 2 Level 2 | landsat_etm_c2_l2 |
| Landsat 8 Collection 2 Level 1 | landsat_ot_c2_l1 |
| Landsat 8 Collection 2 Level 2 | landsat_ot_c2_l2 |
| Landsat 9 Collection 2 Level 1 | landsat_ot_c2_l1 |
| Landsat 9 Collection 2 Level 2 | landsat_ot_c2_l2 |

# Landsat 8/9

### Load Dataset

In [ ]:
# Create a DataFrame from the scenes
df = None
lsat_filename = "metadata/landsat_ot_c2_l1_-50N_products.json"
#lsat_filename = "metadata/landsat_ot_c2_l2_-50N_products.json"
level = 'Level 1' if 'l1' in lsat_filename else 'Level 2'
df = pd.read_json(lsat_filename, orient='index')
df = preprocess_landsat(df, max_date=MAX_DATE)
df = filter_results_with_geojson(df, roi_shape, plot=True)
plot_geometries_on_map(df)
print(list(df))

In [ ]:
print(df['start_time'].min(), df['start_time'].max())

## Summarise Data

In [ ]:
title = f'Landsat 8/9 {level} - Weekly Products' 
plot_timeseries_products(df, title=title,stack_col='sat_id', date_col='start_time',count_freq='7D', plot_freq='1M')

In [ ]:
#year
df_sum = df.groupby(['year']).agg(
    product_count=('landsat_product_id','count'),
    size_mb=('size_compressed_mb','sum'),
    ).reset_index().sort_values('year')
df_sum['size_gb'] = (df_sum['size_mb']/1000).astype(int)
df_sum['size_tb'] = (df_sum['size_gb']/1000).astype(float).round(2)
df_sum

In [ ]:
#year_month
df_sum = df.groupby(['year','month','month_name']).agg(
    product_count=('landsat_product_id','count'),
    size_mb=('size_compressed_mb','sum'),
    ).reset_index().sort_values('month')
df_sum['size_gb'] = (df_sum['size_mb']/1000).astype(int)
df_sum['size_tb'] = (df_sum['size_gb']/1000).astype(int)
df_sum

In [ ]:
# day night
print(df['day-night_indicator'].value_counts())

# sat counts
print('\n')
print(df['satellite'].value_counts())

# cloud cover cats
print(df['cloud_cover_category'].value_counts())


In [ ]:
# sun zenith
df['centroid'] = df['geometry_4326'].apply(lambda x : x.centroid)
df['centroid_x'] = df['centroid'].apply(lambda x : x.x)
df['centroid_y'] = df['centroid'].apply(lambda x : x.y)

bins = [-90, -85, -80, -75, -70, -65, -60, -55, -50, -45]
labels = ['85-90', '80-85', '75-80', '70-75', '65-70', '60-65', '55-60', '50-55', 'less than 50']
df['lon_category'] = pd.cut(x = df['centroid_y'], bins = bins, labels = labels, include_lowest = True)

df['sun_zenith_over_70'] = df['sun_zenith_10ra'] > 70
df['sun_zenith_over_76'] = df['sun_zenith_10ra'] > 76
df['sun_zenith_over_88'] = df['sun_zenith_10ra'] > 88

zenith_by_lat = df[df['year']==2022].groupby('lon_category').agg(
    count=('sun_zenith_over_76','count'),
    sun_zenith_over_70=('sun_zenith_over_70','sum'),
    sun_zenith_over_76=('sun_zenith_over_76','sum'),
    sun_zenith_over_88=('sun_zenith_over_88','sum'),
).reset_index()
zenith_by_lat

#print(df['sun_zenith_over_76'].count(), df['sun_zenith_over_76'].sum())

## Plot Product Count

## Annual

### Day (2022)

In [ ]:
title = f'Landsat 8/9 {level} - Count of Products by Path/Row tile (2022)'
plot_mgrs_product_count(df[(df['year']==2022) & (df['day-night_indicator'] == 'DAY')], title=title)

### Night (2022)

In [ ]:
title = f'Landsat 8/9 {level} - Count of NIGHTTIME Products by Path/Row tile (2022)'
plot_mgrs_product_count(df[(df['year']==2022) & (df['day-night_indicator'] == 'NIGHT')], title=title)

### Monthly (2022)

In [ ]:
# df_tiles = gpd.read_file('WRS2_descending_0_daytime/WRS2_descending.shp') 
# df_tiles
title = f'Landsat 8/9 {level} - Count of Products by Path/Row tile (2022)'
#plot_mgrs_product_count(df, title=title)
plot_multiple_product_count(df[(df['year']==2022) & (df['day-night_indicator'] == 'DAY')], 'month_name', title='', n_cols=3, cbar_label='Count of Products')


### Annual

In [ ]:
title = f'Landsat 8/9 {level} - Revisit Count by Year'
#plot_mgrs_product_count(df, title=title)
plot_multiple_product_count(df[(df['day-night_indicator'] == 'DAY')], 'year', title='', n_cols=4, cbar_label='Revisit Count')


### Cloud Cover Cat

In [ ]:
# df_tiles = gpd.read_file('WRS2_descending_0_daytime/WRS2_descending.shp') 
# df_tiles
title = f'Landsat 8/9 {level} - Count of Products by Cloud Cover Category (2022)'
#plot_mgrs_product_count(df, title=title)
plot_multiple_product_count(df[(df['day-night_indicator'] == 'DAY') & (df['year']==2022) ], 'cloud_cover_category', title=title, n_cols=3, cbar_label='Count of Products')


### Annual (All years)

In [ ]:
# df_tiles = gpd.read_file('WRS2_descending_0_daytime/WRS2_descending.shp') 
# df_tiles
title = f'Landsat 8/9 {level} - Annual Count of Products by Path/Row tile'
#plot_mgrs_product_count(df, title=title)
plot_multiple_product_count(df[(df['day-night_indicator'] == 'DAY')], 'year', title=title, n_cols=4, cbar_label='Count of Products')


## Revisit (2022)

In [ ]:

title = f'Landsat 8/9 {level} Revisit Frequency (2022)'
cbar_label = 'Revisit Count'
intersections = calculate_valid_intersections(df[(df['year']==2022) & (df['day-night_indicator'] == 'DAY')], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='revisit_count', title=title, cbar_label=cbar_label, shape=(1000,1000))

### Monthly (2022)

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 8/9 {level} Revisit Frequency by Month (2022)'
cbar_label = 'Revisit Count'
#intersections = calculate_valid_intersections(df[(df['year']==2022) & (df['day-night_indicator'] == 'DAY')], points_df=points_in_roi)
plot_multiple_intersection_frequency(intersections, plot_var='revisit_count', group='month_name', n_cols=4, title=title, cbar_label=cbar_label, shape=(1000,1000))

In [ ]:
# ### SZA < 76
# # appriximate cloudcover with overlapping products
# title = f'Landsat 8/9 {level} Revisit Frequency SZA < 76 (2022)'
# cbar_label = 'Revisit Count'
# intersections = calculate_valid_intersections(
#     df[(df['year']==2022) & (df['day-night_indicator'] == 'DAY') & (df['sun_zenith_10ra'] < 76)]
#     , points_df=points_in_roi)
# plot_intersection_frequency(intersections, plot_var='revisit_count', title=title, cbar_label=cbar_label, shape=(220,220))

### Cloud Cover (2022)

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 8/9 {level} Approximate Cloud Cover (2022)'
cbar_label = 'Mean Observation Cloud Cover (%)'
#intersections = calculate_valid_intersections(df[df['year']==2022], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='mean_cloudcover', title=title, cbar_label=cbar_label, shape=(1000,1000), cmap='plasma')

### Cloud Cover by Category (2022)

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 8/9 {level} - Count of Observations by Cloud Cover Category (2022)'
cbar_label = 'Revisit Count'
#intersections = calculate_valid_intersections(df[df['year']==2022], points_df=points_in_roi)
plot_multiple_intersection_frequency(intersections, plot_var='revisit_count', group='cloud_cover_category', n_cols=3, title=title, cbar_label=cbar_label, shape=(1000,1000))

### Sun Zenith Angle (2022)

In [ ]:
title = f'Landsat 8/9 {level} - Mean Sun Zenith Angle (2022)'
cbar_label = 'Mean Sun Zenith Angle'
#intersections = calculate_valid_intersections(df[df['year']==2022], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='mean_sza', title=title, cbar_label=cbar_label, shape=(1000,1000))

In [ ]:
title = f'Landsat 8/9 {level} - Minimum Sun Zenith Angle (2022)'
cbar_label = 'Minimum Sun Zenith Angle'
#intersections = calculate_valid_intersections(df[df['year']==2022], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='min_sza', title=title, cbar_label=cbar_label, shape=(1000,1000))

## Revisit 2022 (monthly)

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 8/9 {level} - Revisit Frequency (2022)'
cbar_label = 'Revisit Count'
#intersections = calculate_valid_intersections(df[df['year']==2022], points_df=points_in_roi)
plot_multiple_intersection_frequency(intersections, plot_var='revisit_count', group='month_name', n_cols=4, title=title, cbar_label=cbar_label, shape=(1000,1000))

## Extra stuff

### Product Tier

In [ ]:
df['product_tier'] = df['landsat_product_id'].apply(lambda x : x.split('_')[-1])
df.groupby('product_tier')['landsat_product_id'].count()

### Sun Zenith Azumith (SZA)

In [ ]:
#df['sun_zenith_10ra'] = 90 - df['sun_elevation_l0ra'] 
#df[['sun_elevation_l0ra', 'sun_azimuth_l0ra','sun_zenith_10ra','start_time']]
title = f'Landsat 8/9 {level} Distribution of Product Sun Zenith Angles (2013-03 to 2023-06)' #2013-03-10 22:59:32 2023-06-15 23:28:39
ax = df[(df['day-night_indicator']=='DAY') 
        & (df['year'])]['sun_zenith_10ra'].plot(kind='hist', title=title, bins=20, figsize=(10,5))
ax.axvline(x=76, color='red', linestyle='--')
ax.set_xlabel('Sun Zenith Angle (SZA)')
ax.set_ylabel('Count of Products')
#ax.annotate('Maximum SZA for Producing\nLevel-2 Products (76°)', (77, 14_000), color='red')
ax.annotate('76°', (77, 14_000), color='red')



## Daytime

In [ ]:
print(df['day-night_indicator'].value_counts())
plot_geometries_on_map(df[df['day-night_indicator']=='NIGHT'])


In [ ]:
df[
    (df['wrs_path']==4)
    & (df['wrs_row'].isin([110,111]))
    ].plot(alpha=0.1)
df[
    (df['wrs_path']==4)
    & (df['wrs_row']==110)
    ].shape

## Geometric Accuracy (Heard & Macquarie Islands)

In [ ]:
# Search for Landsat TM scenes
start_date='2013-01-01'
end_date='2023-06-15'

#Heard + mcdonald
loc = 'Heard and McDonald Islands'
bbox = (72,-54,75,-52)
extent = (72, 75, -54, -52)

# Macquarie
# loc = 'Macquarie Island'
# bbox = (157,-56,160,-53)
# extent = (157, 160, -56, -53)

scenes = api.search(
    dataset='landsat_ot_c2_l2',
    bbox=bbox,
    start_date=start_date,
    end_date=end_date,
    max_results=max_results,
)

# Create a DataFrame from the scenes
df = pd.DataFrame(scenes)
df = gpd.GeoDataFrame(df, geometry='spatial_coverage', crs="EPSG:4326")
print(df.shape)
plot_geometries_on_map(df, extent=extent)

In [ ]:
#select which satellite
sat = 8 #landsat 8 or 9 
color = '#1f77b4' if (sat==8) else 'green'
alpha = 1 if ('Heard' in loc) else 1

plot_geometries_on_map(df[~df['geometric_rmse_model'].isna()], extent=extent)
df_control = df[
    (df.ground_control_points_model>0)
    & (df.satellite==sat)]

f, ax =  plt.subplots(1, figsize=(8,6))
ax.hist(df_control.geometric_rmse_model.values, bins=20, color=color, alpha=alpha)
n = len(df_control)
mn,mx = df_control.acquisition_date.min().date(), df_control.acquisition_date.max().date()
ax.set_title(f'Landsat-{sat} Geometric Error - {loc}\n{mn} to {mx} ({n} samples)')
ax.set_xlabel('Geometric RMSE (metres)')
ax.set_ylabel('Number of Samples')
plt.savefig(f'data/landsat{sat}_{loc}_{mn}_{mx}_GCP_acc.png')
plt.show()

df_control = df_control[
    ['entity_id',
    'cloud_cover',
    'acquisition_date',
    'ground_control_points_model',
    'ground_control_points_version',
    'geometric_rmse_model_x',
    'geometric_rmse_model_y',
    'geometric_rmse_model',]].sort_values('acquisition_date')

df_control.to_csv(f'data/landsat{sat}_{loc}_{mn}_{mx}_GPC_acc.csv')


In [ ]:
df_control[['satellite']]

# Landsat 7

## Load Data

In [ ]:
# Create a DataFrame from the scenes
lsat_filename = "metadata/landsat_etm_c2_l1_-50N_products.json"
# lsat_filename = "metadata/landsat_etm_c2_l2_-50N_products.json"
level = 'Level 1' if 'l1' in lsat_filename else 'Level 2'
df = pd.read_json(lsat_filename, orient='index')
df = preprocess_landsat(df, max_date=MAX_DATE)
df = filter_results_with_geojson(df, roi_shape, plot=True)
plot_geometries_on_map(df)
print(list(df))
df.geometry.bounds.miny.min()

In [ ]:
df['start_time'].min()

In [ ]:
title = f'Landsat 7 {level} - Weekly Products' 
plot_timeseries_products(df, title=title,stack_col='sat_id', date_col='start_time',count_freq='7D', plot_freq='1M')

In [ ]:
#year
df_sum = df.groupby(['year']).agg(
    product_count=('landsat_product_id','count'),
    size_mb=('size_compressed_mb','sum'),
    ).reset_index().sort_values('year')
df_sum['size_gb'] = (df_sum['size_mb']/1000).astype(int)
df_sum['size_tb'] = (df_sum['size_gb']/1000).astype(float).round(2)
df_sum

### SZA

In [ ]:
# sun zenith
df['centroid'] = df['geometry_4326'].apply(lambda x : x.centroid)
df['centroid_x'] = df['centroid'].apply(lambda x : x.x)
df['centroid_y'] = df['centroid'].apply(lambda x : x.y)

bins = [-90, -85, -80, -75, -70, -65, -60, -55, -50, -45]
labels = ['85-90', '80-85', '75-80', '70-75', '65-70', '60-65', '55-60', '50-55', 'less than 50']
df['lon_category'] = pd.cut(x = df['centroid_y'], bins = bins, labels = labels, include_lowest = True)

df['sun_zenith_over_76'] = df['sun_zenith'] > 76
df['sun_zenith_over_88'] = df['sun_zenith'] > 88

zenith_by_lat = df[df['year'] ==2022].groupby('lon_category').agg(
    count=('sun_zenith_over_76','count'),
    sun_zenith_over_76=('sun_zenith_over_76','sum'),
    sun_zenith_over_88=('sun_zenith_over_88','sum'),
).reset_index()
zenith_by_lat

print(df['sun_zenith_over_76'].count(), df['sun_zenith_over_76'].sum())

In [ ]:
# NOTE - must be level 1
title = f'Landsat 7 {level} - Distribution of Product Sun Zenith Angles'
ax = df[(df['day-night_indicator']=='DAY')]['sun_zenith'].plot(kind='hist', title=title, bins=20, figsize=(10,5))
ax.axvline(x=76, color='red', linestyle='--')
ax.set_xlabel('Sun Zenith Angle (SZA)')
ax.set_ylabel('Count of Products')
ax.annotate('Maximum SZA for Producing\nLevel-2 Products (76°)', (77, 4_000), color='red')

## Annual (All years)

In [ ]:
title = 'Landsat 7 Level 1 - Count of Products by Path/Row Tile ()'
#plot_mgrs_product_count(df[(df['year']==2011)], title=title)
plot_multiple_product_count(df[df['year']<2015], 'year', title=title, n_cols=6, cbar_label='Count of Products')
#df['year'].value_counts()

## Revisit (2011)

In [ ]:
title = f'Landsat 7 Level 1 Revisit Frequency (2011)' # - ADJUSTED SCALE'
cbar_label = 'Revisit Count'
intersections = calculate_valid_intersections(df[(df['year']==2011) & (df['day-night_indicator'] == 'DAY')], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='revisit_count', title=title, cbar_label=cbar_label, shape=(1000,1000))#, force_max=50)

## Monthly (2011)

In [ ]:
# df_tiles = gpd.read_file('WRS2_descending_0_daytime/WRS2_descending.shp') 
# df_tiles
title = 'Landsat 7 Level 1 - Count of Products by Path/Row tile (2011)'
#plot_mgrs_product_count(df, title=title)
plot_multiple_product_count(df[(df['year']==2011) & (df['day-night_indicator'] == 'DAY')].sort_values('month')
                            , 'month_name', title='', n_cols=3, cbar_label='Count of Products')

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 7 Level 1 Revisit Frequency by month (2011)'
cbar_label = 'Revisit Count'
#intersections = calculate_valid_intersections(df[(df['year']==2011) & (df['day-night_indicator'] == 'DAY')], points_df=points_in_roi)
plot_multiple_intersection_frequency(intersections, plot_var='revisit_count', group='month_name', n_cols=4, title=title, cbar_label=cbar_label, shape=(1000,1000))

# Landsat 5

## Load Data

In [ ]:
# Create a DataFrame from the scenes
lsat_filename = "metadata/landsat_tm_c2_l1_-50N_products.json"
#lsat_filename = "metadata/landsat_tm_c2_l2_-50N_products.json"
level = 'Level 1' if 'l1' in lsat_filename else 'Level 2'
df = pd.read_json(lsat_filename, orient='index')
print(df['satellite'].value_counts())
df = df[df['satellite'].isin(['LANDSAT_4',4])] # exclude lsat 4
df = preprocess_landsat(df)
df = filter_results_with_geojson(df, roi_shape, plot=True)
plot_geometries_on_map(df)
print(list(df))
df.geometry.bounds.miny.min()

In [ ]:
df['start_time'].min(), df['start_time'].max() 

In [ ]:
title = f'Landsat 4/5 {level} - Weekly Products' 
plot_timeseries_products(df, title=title,stack_col='sat_id', date_col='start_time',count_freq='7D', plot_freq='2M')

In [ ]:
#year
df_sum = df.groupby(['year']).agg(
    product_count=('landsat_product_id','count'),
    size_mb=('size_compressed_mb','sum'),
    ).reset_index().sort_values('year')
df_sum['size_gb'] = (df_sum['size_mb']/1000).astype(int)
df_sum['size_tb'] = (df_sum['size_gb']/1000).astype(float).round(2)
df_sum['avg_size_mb'] = df_sum['size_mb']/df_sum['product_count']
df_sum.loc['tot'] = df_sum.sum()
df_sum

In [ ]:
title = f'Landsat 5 {level} - Count of Products by Path/Row Tile'
#plot_mgrs_product_count(df[(df['year']==2011)], title=title)
plot_multiple_product_count(df, 'year', title=title, n_cols=5, cbar_label='Count of Products')
#df['year'].value_counts()

## Revisit 1989

In [ ]:
# appriximate cloudcover with overlapping products
title = f'Landsat 5 {level} Revisit Frequency (1991)'
cbar_label = 'Revisit Count'
intersections = calculate_valid_intersections(df[(df['year']==1991) & (df['day-night_indicator'] == 'DAY')], points_df=points_in_roi)
plot_intersection_frequency(intersections, plot_var='revisit_count', title=title, cbar_label=cbar_label, shape=(1000,1000))

# Filesize

https://www.usgs.gov/faqs/what-are-landsat-collection-1-level-1-data-product-file-sizes

| Sensor | Compressed file | Uncompressed file |
|---|---|---|
| Landsat 8 OLI/TIRS | 919 MB | 1.61 GB |
| Landsat 7 ETM+ | 235 MB | 785 MB |
| Landsat 4-5 TM | 150 MB | 500 MB |
| Landsat 1-5 MSS | 20 MB | 75 MB |